# Rice Grain Classification - 4 Class Model Training
**Classes:** black, brown, chalky, yellow
**Model:** EfficientNet-B0 Transfer Learning

In [3]:
pip install tensorflow

   ---------------------------------------- 0.0/331.7 MB ? eta -:--:--
   ---------------------------------------- 1.8/331.7 MB 11.2 MB/s eta 0:00:30
   ---------------------------------------- 3.9/331.7 MB 10.2 MB/s eta 0:00:33
    --------------------------------------- 6.0/331.7 MB 10.0 MB/s eta 0:00:33
    --------------------------------------- 8.1/331.7 MB 9.7 MB/s eta 0:00:34
   - -------------------------------------- 10.0/331.7 MB 9.7 MB/s eta 0:00:34
   - -------------------------------------- 12.1/331.7 MB 9.6 MB/s eta 0:00:34
   - -------------------------------------- 13.9/331.7 MB 9.6 MB/s eta 0:00:34
   - -------------------------------------- 16.0/331.7 MB 9.5 MB/s eta 0:00:34
   -- ------------------------------------- 18.1/331.7 MB 9.6 MB/s eta 0:00:33
   -- ------------------------------------- 20.2/331.7 MB 9.7 MB/s eta 0:00:33
   -- ------------------------------------- 22.5/331.7 MB 9.8 MB/s eta 0:00:32
   -- ------------------------------------- 24.6/331.7 MB 9.8

In [2]:
# Check if CUDA is available from your system
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

Wed Jan 21 20:11:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.83                 Driver Version: 581.83         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   36C    P3             12W /   70W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
pip install tensorflow[and-cuda]

INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of grpcio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/276.5 MB ? eta -:--:--
   ---------------------------------------- 2.1/276.5 MB 10.7 MB/s eta 0:00:26
    --------------------------------------- 4.2/276.5 MB 10.1 MB/s eta 0:00:28
    --------------------------------------- 6.0/276.5 MB 9.7 MB/s 

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\Adithya Parameswaran\\anaconda3\\envs\\yolov8env\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
# Cell 1: Setup and Imports
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU'))} GPU(s)")

if tf.config.list_physical_devices('GPU'):
    print("GPU acceleration ENABLED!")
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
        print(f"  - {gpu}")
else:
    print("WARNING: No GPU detected. Training will be slow.")

TensorFlow version: 2.20.0
GPU Available: 0 GPU(s)


In [ ]:
# Cell 2: Configuration
BASE_PATH = './rice'  # Local path to rice folder
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE_INITIAL = 0.001
LEARNING_RATE_FINETUNE = 0.0001

# Verify dataset
if os.path.exists(BASE_PATH):
    print(f"Dataset found: {BASE_PATH}")
    for folder in sorted(os.listdir(BASE_PATH)):
        folder_path = os.path.join(BASE_PATH, folder)
        if os.path.isdir(folder_path):
            count = len([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            print(f"  {folder}: {count} images")
else:
    print(f"ERROR: Dataset not found at {BASE_PATH}")

In [ ]:
# Cell 3: Create Datasets
print("Creating datasets...")

# Training + Validation set (80%)
train_val_ds = tf.keras.utils.image_dataset_from_directory(
    BASE_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True
)

# Test set (20%)
test_ds = tf.keras.utils.image_dataset_from_directory(
    BASE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True
)

CLASS_NAMES = train_val_ds.class_names
NUM_CLASSES = len(CLASS_NAMES)

print(f"\nDetected {NUM_CLASSES} classes (ALPHABETICAL ORDER):")
for i, name in enumerate(CLASS_NAMES):
    print(f"  {i}: {name}")

# Split train_val into train and validation
val_batches = max(1, int(0.2 * tf.data.experimental.cardinality(train_val_ds).numpy()))
val_ds = train_val_ds.take(val_batches)
train_ds = train_val_ds.skip(val_batches)

print(f"\nDataset split:")
print(f"  Training batches: {tf.data.experimental.cardinality(train_ds).numpy()}")
print(f"  Validation batches: {tf.data.experimental.cardinality(val_ds).numpy()}")
print(f"  Test batches: {tf.data.experimental.cardinality(test_ds).numpy()}")

In [ ]:
# Cell 4: Preprocessing Functions
def preprocess(image, label):
    """Just cast to float32 - NO normalization (matches inference)"""
    image = tf.cast(image, tf.float32)
    return image, label

def augment(image, label):
    """Data augmentation"""
    if tf.random.uniform([]) > 0.5:
        image = tf.image.random_flip_left_right(image)
    if tf.random.uniform([]) > 0.5:
        image = tf.image.random_flip_up_down(image)
    if tf.random.uniform([]) > 0.6:
        k = tf.random.uniform([], 0, 4, dtype=tf.int32)
        image = tf.image.rot90(image, k=k)
    if tf.random.uniform([]) > 0.7:
        image = tf.image.random_brightness(image, max_delta=0.1 * 255)
        image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    image = tf.clip_by_value(image, 0.0, 255.0)
    return image, label

# Apply preprocessing
train_ds_proc = train_ds.map(preprocess).map(augment).cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds_proc = val_ds.map(preprocess).cache().prefetch(tf.data.AUTOTUNE)
test_ds_proc = test_ds.map(preprocess).cache().prefetch(tf.data.AUTOTUNE)

print("Preprocessing applied!")

In [ ]:
# Cell 5: Calculate Class Weights
print("Calculating class distribution...")
class_counts = np.zeros(NUM_CLASSES, dtype=int)
for _, labels in train_ds_proc:
    for label in labels:
        class_counts[np.argmax(label.numpy())] += 1

total_samples = np.sum(class_counts)
class_weights = {i: total_samples / (NUM_CLASSES * count) if count > 0 else 1.0 
                 for i, count in enumerate(class_counts)}

print(f"\nClass distribution:")
for i, name in enumerate(CLASS_NAMES):
    print(f"  {name}: {class_counts[i]} samples (weight: {class_weights[i]:.2f})")
print(f"  Total: {total_samples} samples")

In [ ]:
# Cell 6: Create Model
print("Building EfficientNet-B0 model...")

base_model = keras.applications.EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMAGE_SIZE, 3)
)
base_model.trainable = False

inputs = keras.Input(shape=(*IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs, name='rice_4class_model')

print(f"Model created with {NUM_CLASSES} output classes")
print(f"Total parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(v).numpy() for v in model.trainable_variables]):,}")

In [ ]:
# Cell 7: PHASE 1 - Train with Frozen Base
print("="*60)
print("PHASE 1: Training with Frozen Base Model")
print("="*60)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_INITIAL),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
    keras.callbacks.ModelCheckpoint('rice_phase1_best.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
]

history1 = model.fit(
    train_ds_proc,
    validation_data=val_ds_proc,
    epochs=EPOCHS//2,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

print(f"\nPhase 1 Complete!")
print(f"  Best Val Accuracy: {max(history1.history['val_accuracy']):.4f}")

In [ ]:
# Cell 8: PHASE 2 - Fine-tuning
print("="*60)
print("PHASE 2: Fine-tuning")
print("="*60)

# Unfreeze last 60 layers of base model
base_model.trainable = True
for layer in base_model.layers[:-60]:
    layer.trainable = False

print(f"Unfrozen layers: {sum(1 for l in base_model.layers if l.trainable)}")

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINETUNE),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=6, min_lr=1e-8),
    keras.callbacks.ModelCheckpoint('rice_phase2_best.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
]

history2 = model.fit(
    train_ds_proc,
    validation_data=val_ds_proc,
    epochs=EPOCHS//2,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

print(f"\nPhase 2 Complete!")
print(f"  Best Val Accuracy: {max(history2.history['val_accuracy']):.4f}")

In [ ]:
# Cell 9: Evaluate and Save
print("="*60)
print("FINAL EVALUATION")
print("="*60)

# Load best model
model = keras.models.load_model('rice_phase2_best.keras')

# Evaluate on test set
test_results = model.evaluate(test_ds_proc, verbose=1)
print(f"\nTest Results:")
print(f"  Loss: {test_results[0]:.4f}")
print(f"  Accuracy: {test_results[1]:.4f}")

# Save final model
model.save('efficientnet_rice_final_inference.keras')
print(f"\nModel saved to: efficientnet_rice_final_inference.keras")

# Save class mapping
print(f"\n" + "="*60)
print("IMPORTANT: Update LABEL_MAP in process_image_updated.py:")
print("="*60)
print("LABEL_MAP = {")
for i, name in enumerate(CLASS_NAMES):
    print(f'    {i}: "{name}",')
print("}")
print(f"\nTraining Complete! Accuracy: {test_results[1]:.4f}")